In [77]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [78]:
from news_summarizer.domain.documents import Article
from news_summarizer.domain.clean_documents import CleanedArticle

In [79]:
import re
import unicodedata

class TextTransformation:
    def apply(self, text: str) -> str:
        raise NotImplementedError

class StripWhitespace(TextTransformation):
    def apply(self, text: str) -> str:
        return text.strip()

class RemoveEmojis(TextTransformation):
    def apply(self, text: str) -> str:
        emoji_pattern = re.compile(
            "["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
            "]+", flags=re.UNICODE)
        return emoji_pattern.sub(r'', text)

class RemoveNonAsciiExceptAccents(TextTransformation):
    def apply(self, text: str) -> str:
        return ''.join(
            c for c in text
            if ord(c) < 128 or unicodedata.category(c).startswith('L')
        )

class ReplaceMultipleSpaces(TextTransformation): 
    def apply(self, text: str) -> str: 
        return re.sub(r'\s+', ' ', text)

class TextPipeline:
    def __init__(self):
        self.transformations = []

    def add_transformation(self, transformation: TextTransformation):
        self.transformations.append(transformation)

    def execute(self, text: str) -> str:
        if text is None:
            return ''
        for transformation in self.transformations:
            text = transformation.apply(text)
        return text

# Example usage
pipeline = TextPipeline()
pipeline.add_transformation(StripWhitespace())
pipeline.add_transformation(RemoveEmojis())
pipeline.add_transformation(RemoveNonAsciiExceptAccents())
pipeline.add_transformation(ReplaceMultipleSpaces())

In [80]:
document_list = list(Article.bulk_find(**{}))

In [81]:
cleaned_documents_list = []

for i, article in enumerate(document_list):
    cleaned_article = CleanedArticle(
        id=article.id,
        title=pipeline.execute(article.title),
        author=article.author,
        content=pipeline.execute(article.content),
        subtitle=pipeline.execute(article.subtitle),
        publication_date=article.publication_date,
        url=article.url
    )

    cleaned_documents_list.append(cleaned_article)

In [82]:
import logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

In [83]:
CleanedArticle.bulk_insert(cleaned_documents_list)

INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/cleaned_articles/points?wait=true "HTTP/1.1 200 OK"


True

In [84]:
CleanedArticle.bulk_find(**{})

INFO:httpx:HTTP Request: POST http://localhost:6333/collections/cleaned_articles/points/scroll "HTTP/1.1 200 OK"


([CleanedArticle(id=UUID('0005fcb9-638b-4695-9319-f472568aad7d'), title='Lula está estável e conversando normalmente após cirurgia, diz médico', subtitle='Presidente teve funções neurológicas preservadas e procedimento decorre do acidente doméstico sofrido em 19 de outubro', author='Plínio Aguiar, do R7, em Brasília', publication_date=datetime.datetime(2024, 12, 10, 13, 15, 11, 788000), content='O presidente Luiz Inácio Lula da Silva tem quadro de saúde estável após a cirurgia realizada na madrugada desta terça-feira (10). De acordo com o médico Roberto Kalil Filho, o petista voltou do procedimento cirúrgico praticamente acordado e conversando normalmente. Não há, por enquanto, data de alta médica, mas a equipe fala em retorno a Brasília no início da próxima semana. Os profissionais clínicos destacam, ainda, que não há lesões cerebrais. O presidente foi submetido a uma cirurgia para drenagem do hematoma do sangramento no cérebro. O presidente evoluiu bem, já chegou da cirurgia praticam